In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json

import lightgbm as lgb
from sklearn.model_selection import GroupKFold

print("Lightgbm version:", lgb.__version__)

import optuna
from optuna.visualization import (
    plot_edf
    , plot_optimization_history
    , plot_parallel_coordinate
    , plot_param_importances
    , plot_slice
)

print("Optuna version:", optuna.__version__)

# local modules
import sys
sys.path.append("../src")
from preproc import process_train_data

Lightgbm version: 4.5.0
Optuna version: 4.0.0


/Users/mavillan/Library/Caches/pypoetry/virtualenvs/mcts-strength-variants-E8z0EJ47-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


***
### load and preprocess data

In [2]:
# define some paths
path_raw = Path("../data/raw")
path_processed = Path("../data/processed")
path_results = Path("../data/results")

# load data
df_train = pd.read_csv(path_raw / "train.csv")
df_test = pd.read_csv(path_raw / "test.csv")

df_train

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,233229,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-NST-false,MCTS-ProgressiveHistory-1.41421356237-Random20...,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",2,0,13,-0.733333
233230,233230,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1-0.6-MAST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",9,1,5,0.266667
233231,233231,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",11,3,1,0.666667
233232,233232,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-true,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",24,2,4,0.666667


In [3]:
# Call the function
df_train, numerical_cols, categorical_cols, encoder, scaler = process_train_data(
    df_train,
    scale=False
)

# Print the results
print("Numerical Columns:", len(numerical_cols))
print("Categorical Columns:", len(categorical_cols))

number of all nan cols:  18
number of constant cols:  198
Numerical Columns: 588
Categorical Columns: 10


***
### optimization



In [4]:
# Define the number of folds for cross-validation
num_folds = 5

# Shuffle the train dataset
df_train = df_train.sample(frac=1, random_state=2112).reset_index(drop=True)

# Define the column for stratified or group k-fold
groups_col = 'GameRulesetName'
gkf = GroupKFold(n_splits=num_folds)
split_list = list(gkf.split(df_train, groups=df_train[groups_col]))

In [5]:
def train_and_score(numerical_cols, categorical_cols):

    # Define the parameters
    params = {
        'objective': "regression",
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'max_depth': 10,
        'learning_rate': 0.1,
        'reg_alpha': 1e-1,
        'reg_lambda': 1e-1,
        'feature_fraction': 0.8,
        'verbose': -1,
        'seed': 2112,
    }

    target = 'utility_agent1'
    oof_scores = []

    # Perform cross-validation
    for _, (train_index, val_index) in enumerate(split_list, 1):
        # Split the data
        X_train, X_val = df_train.iloc[train_index], df_train.iloc[val_index]
        y_train, y_val = X_train[target], X_val[target]
        
        # Create LightGBM datasets
        train_data = lgb.Dataset(
            data=X_train[numerical_cols + categorical_cols],
            label=y_train,
            categorical_feature=categorical_cols,
            free_raw_data=True
        )
        val_data = lgb.Dataset(
            data=X_val[numerical_cols + categorical_cols],
            label=y_val,
            categorical_feature=categorical_cols,
            free_raw_data=True
        )

        # Train the model
        model = lgb.train(
            params,
            train_data,
            num_boost_round=1000,
            valid_sets=[val_data],
        )

        # predict on validation set
        y_pred = model.predict(X_val[numerical_cols + categorical_cols])
        y_pred = np.clip(y_pred, -1, 1)
        
        # Compute RMSE on scaled values
        rmse = np.sqrt(np.mean((y_pred - y_val) ** 2))
        oof_scores.append(rmse)

    return np.mean(oof_scores)


In [6]:
def objective(trial):
    # Feature selection
    selected_numerical = [col for col in numerical_cols if trial.suggest_int(f'use_{col}', 0, 1) == 1]
    selected_categorical = [col for col in categorical_cols if trial.suggest_int(f'use_{col}', 0, 1) == 1]
        
    # Call the train_and_score function with selected features
    score = train_and_score(selected_numerical, selected_categorical)
    
    return score

In [7]:
do_optimize = False
timeout = 3600 * 24

study = optuna.create_study(
    study_name="optuna_lgbm",
    direction='minimize',
    storage='sqlite:///optuna_lgbm.db',
    load_if_exists=True,
)

if do_optimize:

    # manually enqueueing a trial with all features
    # trial_params = {f'use_{col}': 1 for col in numerical_cols + categorical_cols}
    # study.enqueue_trial(trial_params)

    study.optimize(
        objective, 
        n_trials=100000, 
        timeout=timeout,
        n_jobs=1, 
        gc_after_trial=True,
    ) 

[I 2024-10-19 17:58:23,955] Using an existing study with name 'optuna_lgbm' instead of creating a new one.


In [8]:
study.trials_dataframe().sort_values("value", ascending=True).head(20)

,number,value,datetime_start,datetime_complete,duration,params_use_Absolute,params_use_AbsoluteDirections,params_use_AddDecision,params_use_AddDecisionFrequency,params_use_AddEffect,...,params_use_agent1_exploration_const,params_use_agent1_playout,params_use_agent1_score_bounds,params_use_agent1_selection,params_use_agent2,params_use_agent2_exploration_const,params_use_agent2_playout,params_use_agent2_score_bounds,params_use_agent2_selection,state
3542,3542,0.428571,2024-10-19 11:21:19.002427,2024-10-19 11:22:22.828402,0 days 00:01:03.825975,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,COMPLETE
1953,1953,0.428581,2024-10-18 07:57:15.723773,2024-10-18 07:58:15.651364,0 days 00:00:59.927591,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,COMPLETE
1748,1748,0.428581,2024-10-18 04:27:36.280294,2024-10-18 04:28:40.193083,0 days 00:01:03.912789,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,COMPLETE
1750,1750,0.428581,2024-10-18 04:29:44.241007,2024-10-18 04:30:48.759093,0 days 00:01:04.518086,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,COMPLETE
1754,1754,0.428581,2024-10-18 04:34:01.090112,2024-10-18 04:35:06.033235,0 days 00:01:04.943123,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,COMPLETE
1758,1758,0.428581,2024-10-18 04:38:17.793121,2024-10-18 04:39:22.140334,0 days 00:01:04.347213,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,COMPLETE
1759,1759,0.428581,2024-10-18 04:39:22.356660,2024-10-18 04:40:27.014344,0 days 00:01:04.657684,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,COMPLETE
1763,1763,0.428581,2024-10-18 04:43:40.747661,2024-10-18 04:44:45.443596,0 days 00:01:04.695935,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,COMPLETE
1764,1764,0.428581,2024-10-18 04:44:45.657855,2024-10-18 04:45:50.825525,0 days 00:01:05.167670,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,COMPLETE
1765,1765,0.428581,2024-10-18 04:45:51.041671,2024-10-18 04:46:56.282071,0 days 00:01:05.240400,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,COMPLETE


In [9]:
# Get the selected features from the best trial
best_params = study.best_trial.params

# Filter numerical and categorical features
selected_numerical = [col for col in numerical_cols if best_params.get(f'use_{col}', 0) == 1]
selected_categorical = [col for col in categorical_cols if best_params.get(f'use_{col}', 0) == 1]

# Combine selected features
selected_features = selected_numerical + selected_categorical

print("Selected features:")
print("len(numerical): ", len(selected_numerical), "/", len(numerical_cols))
print("len(categorical): ", len(selected_categorical), "/", len(categorical_cols))
print("len(selected_features): ", len(selected_features), "/", len(numerical_cols) + len(categorical_cols))


# Save selected features to a file
selected_features_dict = {
    "numerical": selected_numerical,
    "categorical": selected_categorical
}
with open('select_optuna_lgbm.json', 'w') as f:
    json.dump(selected_features_dict, f, indent=2)

print("Selected features have been saved to 'select_optuna_lgbm.json'")

Selected features:
len(numerical):  281 / 588
len(categorical):  6 / 10
len(selected_features):  287 / 598
Selected features have been saved to 'select_optuna_lgbm.json'


***